In [9]:
model_path = "../BirdNET/models/CustomClassifier_Append.tflite"

In [10]:
# import tensorflow as tf
# import numpy as np

# # Load the TFLite model
# interpreter = tf.lite.Interpreter(model_path=model_path)
# interpreter.allocate_tensors()

# # Get layer details
# layer_details = interpreter.get_tensor_details()

# # Print layer details
# for layer in layer_details:
#     print("Layer Name:", layer['name'])
#     print("Layer Index:", layer['index'])
#     print("Layer Shape:", layer['shape'])
#     print("Layer Type:", layer['dtype'])
#     print("Quantization Parameters:", layer['quantization'])
#     print("-" * 30)


In [11]:
# # Get input and output tensors
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Print model summary
# print("=== Model Summary ===")
# print("Input details:")
# for detail in input_details:
#     print(detail)
# print("\nOutput details:")
# for detail in output_details:
#     print(detail)

In [12]:
# interpreter = tf.lite.Interpreter(model_path=model_path)
# interpreter.allocate_tensors()

# # Obtener los índices de entrada y salida
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Configurar los índices de la capa de embedding (en este caso, la capa GlobalAveragePooling)
# embedding_index = 550 #547  # Índice de model/GLOBAL_AVG_POOL/Mean, como identificaste

# # Asignar una entrada de prueba
# input_data = np.random.random(size=input_details[0]['shape']).astype(np.float32)
# interpreter.set_tensor(input_details[0]['index'], input_data)

# # Ejecutar la inferencia
# interpreter.invoke()

# # Extraer el embedding
# embedding = interpreter.get_tensor(embedding_index)
# print("Embedding shape:", embedding.shape)
# print("Embedding vector:", embedding)

In [13]:
# 8_CustomClassifier_Append -> JUST BIRDNET
# predict_test_model12_conf015 -> BIDSSONGDETECTOR+BIRDNET

In [14]:
# # save embeddings
# np.save("embeddings.npy", embedding)
# print("Embeddings saved to embeddings.npy")

## Embeddings Extraction

### Train

In [15]:
import os
import numpy as np
import tensorflow as tf
import librosa

# Cargar el modelo TFLite
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Configuración de audio
sample_rate = 32000  # Tasa de muestreo original
target_sample_rate = 48000  # Tasa de muestreo objetivo para el modelo
target_duration = 3  # Duración objetivo en segundos
target_length = target_sample_rate * target_duration  # Longitud en frames para 3 segundos

# Función para preprocesar cada audio
def preprocess_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=sample_rate)
    if sr != target_sample_rate:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sample_rate)
    if len(audio) > target_length:
        start = (len(audio) - target_length) // 2
        audio = audio[start:start + target_length]
    else:
        padding = target_length - len(audio)
        audio = np.pad(audio, (padding // 2, padding - padding // 2), 'constant')
    return audio

# Función para obtener el embedding desde el modelo TFLite
def get_embedding(audio_data):
    input_shape = input_details[0]['shape']
    audio_data = np.reshape(audio_data, input_shape).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], audio_data)
    interpreter.invoke()
    embedding = interpreter.get_tensor(output_details[0]['index'])
    return embedding.flatten()

# Obtener embeddings y etiquetas del conjunto de entrenamiento
train_folder = "../BirdNET/Audios/train"
train_embeddings = []
train_labels = []

for label_folder in os.listdir(train_folder):
    label_path = os.path.join(train_folder, label_folder)
    if os.path.isdir(label_path):
        for file_name in os.listdir(label_path):
            if file_name.endswith(".WAV"):
                audio_path = os.path.join(label_path, file_name)
                audio_data = preprocess_audio(audio_path)
                embedding = get_embedding(audio_data)
                train_embeddings.append(embedding)
                train_labels.append(label_folder)  # La etiqueta es el nombre de la carpeta

train_embeddings = np.array(train_embeddings)
train_labels = np.array(train_labels)


### Test Embeddings

In [16]:
# Obtener embeddings para el conjunto de prueba
test_folder = "../BirdNET/Audios/predict_test_model12_conf015"
test_embeddings = []
test_file_names = []

for file_name in os.listdir(test_folder):
    if file_name.endswith(".WAV"):
        audio_path = os.path.join(test_folder, file_name)
        audio_data = preprocess_audio(audio_path)
        embedding = get_embedding(audio_data)
        test_embeddings.append(embedding)
        test_file_names.append(file_name)

test_embeddings = np.array(test_embeddings)

## Train Classifier with embeddings

In [19]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train_labels)

# Dividir los datos de entrenamiento en conjuntos de entrenamiento y validación
# X_train, X_val, y_train, y_val = train_test_split(train_embeddings, encoded_labels, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
# rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_clf.fit(train_embeddings, encoded_labels)

# XGBoost
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(train_embeddings, encoded_labels)

# # Validación
# y_val_pred = rf_clf.predict(X_val)
# print("Precisión en validación:", accuracy_score(y_val, y_val_pred))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

## Load and preprocess Audios

In [21]:
# Predecir etiquetas de los audios de prueba
# test_predictions = rf_clf.predict(test_embeddings)

# xgboost
test_predictions = xgb_clf.predict(test_embeddings)

# Decodificar las etiquetas predichas
test_predictions = label_encoder.inverse_transform(test_predictions)

# Mostrar las predicciones para cada archivo
for file_name, pred in zip(test_file_names, test_predictions):
    print(f"Archivo: {file_name}, Predicción: {pred}")

Archivo: AM10_20230523_063000_11.74_13.34_0.40.WAV, Predicción: Background
Archivo: AM10_20230718_071000_32.27_34.92_0.20.WAV, Predicción: Luscinia megarhynchos_Common Nightingale
Archivo: AM1_20230510_083000_17.69_21.92_0.44.WAV, Predicción: Galerida theklae_Thekla's Lark
Archivo: AM10_20230718_071000_56.82_57.83_0.15.WAV, Predicción: Linaria cannabina_Eurasian Linnet
Archivo: AM1_20230514_103000_9.53_10.70_0.39.WAV, Predicción: Galerida theklae_Thekla's Lark
Archivo: AM8_20230523_064000_52.48_53.54_0.17.WAV, Predicción: Sturnus unicolor_Spotless Starling
Archivo: AM1_20230511_093000_59.07_59.98_0.18.WAV, Predicción: Galerida theklae_Thekla's Lark
Archivo: AM11_20230530_065000_16.24_20.75_0.19.WAV, Predicción: Melanocorypha calandra_Calandra Lark
Archivo: AM8_20230523_064000_25.49_27.06_0.33.WAV, Predicción: Emberiza calandra_Corn Bunting
Archivo: AM10_20230718_093000_49.88_51.67_0.59.WAV, Predicción: Background
Archivo: AM1_20230512_103000_2.69_4.92_0.45.WAV, Predicción: Milvus migra

In [24]:
# Save predictions in a folder named "predictions" for each prediction, one file with the name of the audio and the prediction, start_time = 0 and end_time = 3
predictions_folder = "BirdSongDetectorRGBPredictions"
os.makedirs(predictions_folder, exist_ok=True)

for file_name, pred in zip(test_file_names, test_predictions):
    filename = file_name.replace("WAV", "BirdNET.results")
    with open(os.path.join(predictions_folder, filename + ".csv"), "w") as f:
        # first line is header: Start (s),End (s),Scientific name,Common name,Confidence
        f.write("Start (s),End (s),Scientific name,Common name,Confidence\n")
        if pred != "Background":
            # if pred contains _ then split it and use the first part as scientific name and the second part as common name
            if "_" in pred:
                scientific_name, common_name = pred.split("_", 1)
            else:
                scientific_name = pred
                common_name = pred
            f.write(f"0,3.0,{scientific_name},{common_name},1\n")

print("Predictions saved to the 'predictions' folder")

Predictions saved to the 'predictions' folder


In [ ]:
# print all csv filename in predict_test_model12_conf015 sorted alphabetically
for file_name in sorted(os.listdir(test_folder)):
    if file_name.endswith(".WAV"):
        print(file_name)

AM10_20230509_063000_21.77_23.40_0.18.WAV
AM10_20230509_090000_0.01_1.03_0.16.WAV
AM10_20230509_090000_1.79_3.22_0.19.WAV
AM10_20230509_090000_13.00_14.38_0.20.WAV
AM10_20230509_090000_19.78_20.83_0.16.WAV
AM10_20230509_090000_23.78_25.38_0.16.WAV
AM10_20230509_090000_31.95_32.46_0.22.WAV
AM10_20230509_090000_36.93_38.23_0.15.WAV
AM10_20230509_090000_49.04_50.29_0.18.WAV
AM10_20230509_090000_5.47_6.95_0.30.WAV
AM10_20230509_090000_52.85_53.90_0.28.WAV
AM10_20230509_090000_58.66_59.97_0.19.WAV
AM10_20230523_063000_0.00_0.88_0.17.WAV
AM10_20230523_063000_11.74_13.34_0.40.WAV
AM10_20230523_063000_18.33_19.56_0.62.WAV
AM10_20230523_063000_2.42_3.88_0.18.WAV
AM10_20230523_063000_24.40_25.89_0.73.WAV
AM10_20230523_063000_30.41_31.32_0.23.WAV
AM10_20230523_063000_32.95_34.51_0.79.WAV
AM10_20230523_063000_45.84_47.01_0.18.WAV
AM10_20230523_063000_5.74_6.93_0.30.WAV
AM10_20230523_063000_50.13_51.67_0.55.WAV
AM10_20230523_063000_51.25_53.21_0.25.WAV
AM10_20230523_063000_53.93_55.37_0.45.WAV
AM10

In [43]:
!ls ../Data/Dataset/Audios/test/

 AM1   AM10   AM11   AM15   AM4   AM8  'Data Augmentation'


In [23]:
segment_duration = 3  # Duración de cada segmento en segundos

# Función para preprocesar y ajustar un segmento de audio
def preprocess_segment(audio_segment):
    if len(audio_segment) < target_length:
        padding = target_length - len(audio_segment)
        audio_segment = np.pad(audio_segment, (0, padding), 'constant')
    return audio_segment[:target_length]

# Función para dividir el audio en segmentos de 3 segundos
def split_audio_into_segments(audio_path):
    audio, sr = librosa.load(audio_path, sr=sample_rate)
    segments = []
    for start in range(0, len(audio), target_length):
        segment = audio[start:start + target_length]
        segment = preprocess_segment(segment)
        segments.append(segment)
    return segments

# Carpeta de audios de prueba organizados en subcarpetas
test_folder = "../Data/Dataset/Audios/test/"

# Carpeta de salida para las predicciones
# predictions_folder = "RFPredictions"
predictions_folder = "RGBPredictions"
os.makedirs(predictions_folder, exist_ok=True)

# Procesar cada archivo de audio en cada subcarpeta
for root, _, files in os.walk(test_folder):
    for file_name in files:
        if file_name.endswith(".WAV"):
            audio_path = os.path.join(root, file_name)
            segments = split_audio_into_segments(audio_path)

            # Obtener embeddings y predicciones para cada segmento
            segment_predictions = []
            for i, segment in enumerate(segments):
                embedding = get_embedding(segment)
                # prediction = rf_clf.predict([embedding])[0]  # Predecir usando RandomForest
                prediction = xgb_clf.predict([embedding])[0]

                # decode prediction
                prediction = label_encoder.inverse_transform([prediction])[0]
                
                # Solo agregar predicción si no es "background"
                if prediction != "Background":
                    start_time = i * segment_duration
                    end_time = start_time + segment_duration
                    segment_predictions.append((start_time, end_time, prediction))

            # Guardar predicciones en un archivo CSV replicando la estructura de carpetas
            relative_path = os.path.relpath(audio_path, test_folder)
            output_path = os.path.join(predictions_folder, os.path.dirname(relative_path))
            os.makedirs(output_path, exist_ok=True)
            output_filename = os.path.splitext(file_name)[0] + ".BirdNET.results.csv"
            with open(os.path.join(output_path, output_filename), "w") as f:
                f.write("Start (s),End (s),Scientific name,Common name,Confidence\n")
                for start, end, pred in segment_predictions:
                    if "_" in pred:
                        scientific_name, common_name = pred.split("_", 1)
                    else:
                        scientific_name = common_name = pred
                    f.write(f"{start},{end},{scientific_name},{common_name},1\n")

print("Predictions saved to the 'RFPredictions' folder")

Predictions saved to the 'RFPredictions' folder
